## what is union find?

## implement and test union-find in python

## imports

In [17]:
import numpy as np
from time import time as m_time


## union-find

simple union-find implementation

testing some variants:
1. make_uf: list based
1. make_ufw: list+rank admin
1. make_ufnp: numpy int array
1. make_ufex: external root

In [46]:
def make_uf(N):
  root=[term]*N
# compressing
  def find_(i):
    ans=i
    while root[ans]!=(-1):
      ans=root[ans]
    while root[i]!=(-1):
      root[i],i=ans,root[i]
    return ans
# simple union  
  def union_(i,j):
    i,j=find_(i),find_(j)
    if i!=j:
      root[j]=i
      return 1
    return 0
  return find_,union_

def make_ufw(N):
  root=[-1]*N
  w=[1]*N
# compressing
  def find_(i):
    ans=i
    while root[ans]!=(-1):
      ans=root[ans]
    while root[i]!=(-1):
      root[i],i=ans,root[i]
    return ans
# simple union  
  def union_(i,j):
    i,j=find_(i),find_(j)
    if i!=j:
      if w[j]>w[i]: i,j=j,i
      root[j]=i
      w[i],w[j]=w[i]+w[j],0
      return 1
    return 0
  return find_,union_

def make_ufnp(N):
  root=np.full(N,-1)
# compressing
  def find_(i):
    nonlocal root
    ans=i
    while root[ans]+1>0:
      ans=root[ans]
    while root[i]+1>0:
      root[i],i=ans,root[i]
    return ans
# simple union  
  def union_(i,j):
    nonlocal root
    i,j=find_(i),find_(j)
    if i!=j:
      root[j]=i
      return 1
    return 0
  return find_,union_

def make_ufex(root):
# compressing
  def find_(i):
    ans=i
    while root[ans]>=0:
      ans=root[ans]
    while root[i]>=0:
      root[i],i=ans,root[i]
    return ans
# simple union  
  def union_(i,j):
    i,j=find_(i),find_(j)
    if i!=j:
      root[j]=i
      return 1
    return 0
  return find_,union_



### small test

In [49]:
# small tests
find,uni=make_uf(5)
print(find(0)," ",find(1))
uni(0,1)
print(find(0)," ",find(1))
uni(1,2)
uni(3,4)
print([find(i) for i in range(5)])
uni(2,3)
[find(i) for i in range(5)]


0   1
0   0
[0, 0, 0, 3, 3]


[0, 0, 0, 0, 0]

## measure exec time

In [32]:
N=5000000
s=m_time()
a=[0]*N
print(m_time()-s)

s=m_time()
b=[0 for i in range(N)] # several times slower
print(m_time()-s)
a,b=[],[]

0.020572662353515625
0.2789781093597412


### a simple tic() function

In [21]:
def make_tic():
  s0=m_time()
  def tic():
    nonlocal s0
    delta=m_time()-s0
    s0+=delta
    return delta
  return tic
tic=make_tic()
type(tic)

function

#### testing tic()

In [22]:
tic()
a=list(range(10))
print(tic())
a=list(range(100))
print(tic())
a=list(range(1000))
print(tic())
a=list(range(100000))
print(tic())



8.535385131835938e-05
0.0005965232849121094
0.00015687942504882812
0.0041046142578125


## testing tuple assignment

In [23]:
# eloszor a jobb oldal (balrol-jobbra), majd a baloldal
a=list(range(3))
i=0
i,a[i]=i+1,a[i]+10
print(a)

i=0
a=list(range(3))
a[i],i=a[i]+1,i+1
print(a)


[0, 10, 2]
[1, 1, 2]


## slices

In [24]:
a=list(range(10))
print(a[-1::-1])
print(a[:3])

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
[0, 1, 2]


## random numbers

In [25]:
import random as rnd
print(rnd.seed(42)) # 
print(rnd.randint(0,1))
print(rnd.random())

#rnd.sample(range(10),10)

None
0
0.025010755222666936


### random sample

In [60]:
tic()
N=100000
M=100000
m_edges=[]
m_pop=range(N)
E=range(M)
for i in E:
  m_edges.append(rnd.sample(m_pop,2))
print(tic())



0.8718576431274414


## union-find: a larger test

In [64]:
tic()
_,uni=make_uf(N)
NC=N
for e in m_edges:
  x,y=e
  NC-=uni(x,y)
print(NC)  
print("uf -1",tic())

tic()
_,u2=make_uf(N)
NC=N
for e in m_edges:
  x,y=e
  NC-=u2(x,y)
print(NC)  
print("uf None",tic())


# tic()
# _,u3=make_ufnp(N)
# NC=N
# for e in m_edges:
#   x,y=e
#   NC-=u3(x,y)
# print(NC)  
# print("ufnp",tic())

# tic()
# _,u4=make_ufex(np.full(N,-1))
# NC=N
# for e in m_edges:
#   x,y=e
#   NC-=u4(x,y)
# print(NC)  
# print("ufex",tic())



16271
uf -1 0.3198552131652832
16271
uf None 0.2973799705505371


## class based union-find

In [43]:
# Python3 port of http://algs4.cs.princeton.edu/code/edu/princeton/cs/algs4/UF.java.html
class DisjointSet:
  """
  Execution:    python disjoint_set.py < input.txt
  Data files:   http://algs4.cs.princeton.edu/15uf/tinyUF.txt
  http://algs4.cs.princeton.edu/15uf/mediumUF.txt
  http://algs4.cs.princeton.edu/15uf/largeUF.txt

  Weighted quick-union by rank with path compression.

  % python disjoint_set.py < tinyUF.txt
  4 3
  3 8
  6 5
  9 4
  2 1
  5 0
  7 2
  6 1
  2 components
  """
  def __init__(self, n):
    """Initializes an empty union–find data structure with n sites
    0 through n-1. Each site is initially in its own component.

    Args:
    n (int): the number of sites
    """
    # number of components
    self.count = n 
    # parent[i] = parent of i
    self.parent = list(range(n))
    # rank[i] = rank of subtree rooted at i (never more than 31)
    self.rank = [0] * n  

  def find(self, p):
    """Returns the component identifier for the component 
    containing site p.

    Args:
    p (int): the integer representing one site

    Returns:
    int. the component identifier for the component containing site p
    """
    #self.validate(p)
    if p != self.parent[p]:
      self.parent[p] = self.find(self.parent[p]) # path compression
    return self.parent[p]


  def connected(self, p, q):
    """Returns true if the the two sites are in the same component.

    Args:
    p (int): the integer representing one site
    q (int): the integer representing the other site

    Returns:
    Boolean. true if the two sites p and q are in the same component;
    false otherwise.
    """
    return self.find(p) == self.find(q)

  def union(self, p, q):
    """Merges the component containing site p with the 
    the component containing site q.

    Args:
    p (int): the integer representing one site
    q (int): the integer representing the other site
    """
    rootP = self.find(p)
    rootQ = self.find(q)
    if rootP == rootQ: return

    # make root of smaller rank point to root of larger rank
    if   self.rank[rootP] < self.rank[rootQ]: self.parent[rootP] = rootQ
    elif self.rank[rootP] > self.rank[rootQ]: self.parent[rootQ] = rootP
    else:
      self.parent[rootQ] = rootP
      self.rank[rootP] += 1
    self.count -= 1

  def validate(self, p):
    """validate that p is a valid index"""
    n = len(self.parent)
    if p < 0 or p >= n:
      raise IndexError("index {} is not between 0 and {}".format(p, n - 1)) 



### test it

In [44]:
tic()
ds = DisjointSet(N)
for x in m_edges:
  p, q = x
  if ds.connected(p, q):
    continue
  ds.union(p, q)
#print(p, q)
print(ds.count, "components")
print(tic())

1 components
7.0779712200164795
